In [1]:
import pandas as pd
import xgboost as xgb
import numpy as np

In [3]:
data_23 = pd.read_csv('data/pl2022-23.csv')
data_24 = pd.read_csv('data/pl2023-24.csv')
data_25 = pd.read_csv('data/pl2024-25.csv')

In [4]:
data_23.describe()

,TeamID,GoalsForHome,GoalsAgainstHome,GoalsForAway,GoalsAgainstAway,WinsHome,DrawsHome,LossesHome,WinsAway,DrawsAway,...,Losses,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Rank,last_5_winners,Rating
count,20.00000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,...,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.00000,20.000000,20.000000
mean,10.50000,31.050000,23.150000,23.150000,31.050000,9.200000,4.350000,5.450000,5.450000,4.350000,...,14.650000,52.650000,1.385500,53.890000,53.885000,-0.005000,-0.001000,10.50000,0.250000,6.850000
std,5.91608,11.869798,6.784464,7.713795,8.300127,3.833371,1.926956,3.252125,2.946452,1.926956,...,5.993637,18.189861,0.477829,13.762875,9.518916,21.949091,0.578891,5.91608,0.910465,0.098408
min,1.00000,16.000000,10.000000,11.000000,16.000000,2.000000,1.000000,1.000000,1.000000,2.000000,...,5.000000,25.000000,0.660000,36.800000,32.100000,-25.300000,-0.670000,1.00000,0.000000,6.690000
25%,5.75000,20.750000,18.750000,17.750000,25.750000,6.000000,3.000000,2.000000,3.750000,3.000000,...,9.000000,38.750000,1.022500,43.725000,49.400000,-19.950000,-0.525000,5.75000,0.000000,6.785000
50%,10.50000,29.000000,23.500000,22.000000,30.500000,8.500000,4.000000,5.500000,5.000000,4.000000,...,15.500000,48.500000,1.275000,49.900000,52.500000,-3.450000,-0.090000,10.50000,0.000000,6.845000
75%,15.25000,36.250000,27.000000,29.750000,38.000000,12.000000,6.000000,7.000000,7.250000,5.000000,...,19.250000,63.250000,1.662500,68.650000,63.375000,18.150000,0.475000,15.25000,0.000000,6.930000
max,20.00000,60.000000,37.000000,35.000000,44.000000,17.000000,7.000000,12.000000,12.000000,9.000000,...,25.000000,89.000000,2.340000,78.600000,67.100000,46.500000,1.220000,20.00000,4.000000,7.070000


In [5]:
data_24.describe()

,TeamID,GoalsForHome,GoalsAgainstHome,GoalsForAway,GoalsAgainstAway,WinsHome,DrawsHome,LossesHome,WinsAway,DrawsAway,...,Losses,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Rank,last_5_winners,Rating
count,20.00000,20.000000,20.00000,20.000000,20.000000,20.00000,20.000000,20.000000,20.000000,20.000000,...,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,2.000000e+01,20.00000,20.000000,19.000000
mean,10.50000,34.200000,28.10000,28.100000,34.200000,8.75000,4.100000,6.150000,6.150000,4.100000,...,14.900000,52.300000,1.376000,58.870000,58.880000,-0.010000,-5.551115e-18,10.50000,0.250000,6.959474
std,5.91608,10.551129,9.62945,7.919995,8.666937,3.98517,1.803505,3.602265,3.232646,1.586124,...,6.512336,20.596627,0.542338,14.066537,12.508633,24.454574,6.441477e-01,5.91608,0.910465,0.136320
min,1.00000,19.000000,16.00000,16.000000,13.000000,2.00000,0.000000,0.000000,1.000000,2.000000,...,3.000000,16.000000,0.420000,38.300000,27.900000,-38.300000,-1.010000e+00,1.00000,0.000000,6.670000
25%,5.75000,27.000000,23.50000,23.500000,31.750000,6.50000,3.000000,3.750000,4.000000,3.000000,...,11.750000,39.750000,1.045000,49.575000,54.725000,-14.075000,-3.700000e-01,5.75000,0.000000,6.890000
50%,10.50000,31.000000,27.50000,26.500000,35.000000,8.00000,4.000000,6.000000,5.500000,4.000000,...,14.500000,48.500000,1.275000,56.200000,59.000000,-1.700000,-4.500000e-02,10.50000,0.000000,6.940000
75%,15.25000,45.000000,30.00000,33.750000,37.500000,12.00000,5.000000,8.000000,7.250000,5.250000,...,18.250000,63.750000,1.680000,69.775000,68.000000,7.250000,1.925000e-01,15.25000,0.000000,7.020000
max,20.00000,51.000000,57.00000,45.000000,48.000000,15.00000,8.000000,13.000000,14.000000,7.000000,...,28.000000,91.000000,2.390000,87.800000,78.000000,48.200000,1.270000e+00,20.00000,4.000000,7.310000


In [6]:
data_25.describe()

,TeamID,GoalsForHome,GoalsAgainstHome,GoalsForAway,GoalsAgainstAway,WinsHome,DrawsHome,LossesHome,WinsAway,DrawsAway,...,Losses,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Rank,last_5_winners,Rating
count,20.00000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.00000,20.000000,20.000000,...,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.00000,20.000000,20.000000
mean,10.50000,28.750000,27.000000,27.000000,28.750000,7.750000,4.650000,6.60000,6.600000,4.650000,...,14.350000,52.350000,1.378500,53.905000,53.890000,-0.005000,0.001000,10.50000,0.250000,6.926000
std,5.91608,9.106822,9.250889,7.276494,7.122056,3.683177,1.899446,4.15996,2.562893,1.663066,...,6.960603,18.576372,0.488578,13.037617,12.002978,23.293719,0.613033,5.91608,0.910465,0.102926
min,1.00000,13.000000,16.000000,13.000000,17.000000,1.000000,2.000000,1.00000,1.000000,2.000000,...,4.000000,12.000000,0.320000,32.600000,34.400000,-52.100000,-1.370000,1.00000,0.000000,6.730000
25%,5.75000,23.000000,19.500000,22.750000,24.750000,5.750000,3.000000,3.00000,5.750000,3.750000,...,9.750000,42.000000,1.110000,45.050000,47.275000,-6.550000,-0.172500,5.75000,0.000000,6.885000
50%,10.50000,27.000000,26.000000,27.000000,27.500000,7.500000,4.000000,6.00000,7.000000,4.500000,...,12.000000,55.000000,1.445000,57.400000,49.600000,2.700000,0.070000,10.50000,0.000000,6.920000
75%,15.25000,35.000000,34.000000,29.750000,31.500000,11.000000,5.250000,9.25000,8.000000,6.000000,...,18.500000,66.000000,1.740000,61.250000,58.500000,16.200000,0.427500,15.25000,0.000000,6.982500
max,20.00000,43.000000,47.000000,44.000000,46.000000,14.000000,9.000000,15.00000,11.000000,8.000000,...,30.000000,84.000000,2.210000,82.200000,84.800000,43.600000,1.150000,20.00000,4.000000,7.130000


In [7]:
data_23 = data_23.drop(columns=['TeamID'])
data_24 = data_24.drop(columns=['TeamID'])
data_25 = data_25.drop(columns=['TeamID'])

data = pd.concat([data_24, data_25,data_23], ignore_index=True)

data.describe()

,GoalsForHome,GoalsAgainstHome,GoalsForAway,GoalsAgainstAway,WinsHome,DrawsHome,LossesHome,WinsAway,DrawsAway,LossesAway,...,Losses,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Rank,last_5_winners,Rating
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,...,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,6.000000e+01,60.000000,60.000000,59.000000
mean,31.333333,26.083333,26.083333,31.333333,8.566667,4.366667,6.066667,6.066667,4.366667,8.566667,...,14.633333,52.433333,1.380000,55.555000,55.551667,-0.006667,3.700743e-18,10.500000,0.250000,6.911017
std,10.630412,8.764899,7.809798,8.233184,3.819516,1.859166,3.658594,2.916154,1.716981,3.238365,...,6.393765,18.823324,0.495112,13.603146,11.471505,22.857457,6.021318e-01,5.814943,0.894901,0.120583
min,13.000000,10.000000,11.000000,13.000000,1.000000,0.000000,0.000000,1.000000,2.000000,2.000000,...,3.000000,12.000000,0.320000,32.600000,27.900000,-52.100000,-1.370000e+00,1.000000,0.000000,6.670000
25%,23.000000,19.750000,20.750000,25.000000,6.000000,3.000000,3.000000,4.000000,3.000000,7.000000,...,10.000000,39.750000,1.045000,46.025000,48.725000,-15.225000,-4.000000e-01,5.750000,0.000000,6.840000
50%,30.000000,26.000000,26.000000,31.000000,8.000000,4.000000,6.000000,6.000000,4.000000,8.000000,...,14.000000,52.000000,1.370000,54.950000,54.900000,-1.750000,-4.500000e-02,10.500000,0.000000,6.920000
75%,37.000000,30.000000,32.000000,37.000000,12.000000,5.250000,8.250000,8.000000,5.000000,11.000000,...,19.000000,66.000000,1.740000,64.925000,63.325000,15.725000,4.150000e-01,15.250000,0.000000,6.970000
max,60.000000,57.000000,45.000000,48.000000,17.000000,9.000000,15.000000,14.000000,9.000000,15.000000,...,30.000000,91.000000,2.390000,87.800000,84.800000,48.200000,1.270000e+00,20.000000,4.000000,7.310000


In [8]:
teams = data['Team'].unique()
teams_dict = {team: i+1 for i, team in enumerate(teams)}
teams_dict

{'Burnley': 1,
 'Arsenal': 2,
 'Bournemouth': 3,
 'Brighton': 4,
 'Everton': 5,
 'Sheffield United': 6,
 'Newcastle': 7,
 'Brentford': 8,
 'Chelsea': 9,
 'Man United': 10,
 "Nott'm Forest": 11,
 'Fulham': 12,
 'Liverpool': 13,
 'Wolves': 14,
 'Tottenham': 15,
 'Man City': 16,
 'Aston Villa': 17,
 'West Ham': 18,
 'Crystal Palace': 19,
 'Luton': 20,
 'Ipswich': 21,
 'Leicester': 22,
 'Southampton': 23,
 'Leeds': 24}

In [9]:
data['Team'] = data['Team'].map(teams_dict)

In [10]:
# Concatenate training data from 23 and 24 seasons
X_train = pd.concat([data_23, data_24], ignore_index=True)
X_train = X_train.drop(columns=['Rank'])

# Prepare test data from 25 season (drop target column)
X_test = data_25.drop(columns=['Rank'])

# Concatenate training labels from 23 and 24 seasons
y_train = pd.concat([data_23["Rank"], data_24["Rank"]], ignore_index=True)

# Test labels from 25 season
y_test = data_25["Rank"]


In [11]:
X_train["Team"] = X_train["Team"].map(teams_dict)
X_test["Team"] = X_test["Team"].map(teams_dict)

In [12]:
pos = sum(y_train == 1)
neg = sum(y_train == 0)
scale_pos_weight = neg / pos

model = xgb.XGBRegressor(
    max_depth=4,
    n_estimators=200,
    random_state=42
)
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=4,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=200,
             n_jobs=None, num_parallel_tree=None, ...)

In [13]:
predicted_ranks = model.predict(X_test)

In [17]:
# Invert ranks to "scores" (higher score = better chance)
inv_ranks = 1 / predicted_ranks

# Normalize to get probabilities summing to 1
win_probs = inv_ranks / np.sum(inv_ranks)

# Add to test dataframe for review
result = data_25.copy()
result['PL_Champions'] = win_probs *100
ranks = np.arange(1,21)
result = result.sort_values(by='PL_Champions', ascending=False)
result['PL_Champions'] = result['PL_Champions'].round(2).astype(str) + '%'
result['Predicted_Rank'] = ranks

print(result[['Team','PL_Champions', 'Predicted_Rank']])

              Team PL_Champions  Predicted_Rank
19       Liverpool       11.75%               1
2          Arsenal       11.68%               2
13        Man City        8.97%               3
8          Chelsea         8.8%               4
4        Newcastle        7.35%               5
16     Aston Villa        6.98%               6
5    Nott'm Forest        6.12%               7
10        Brighton        4.01%               8
7        Brentford        3.84%               9
11  Crystal Palace         3.8%              10
12          Fulham        3.78%              11
17     Bournemouth        3.72%              12
3          Everton        3.24%              13
6         West Ham        2.82%              14
18          Wolves        2.66%              15
0       Man United        2.64%              16
15       Tottenham        2.33%              17
9        Leicester        1.87%              18
1          Ipswich        1.87%              19
14     Southampton        1.78%         